# ProtoNN in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of ProtoNN. We are using the USPS dataset. Please refer to `fetch_usps.py` for more details on downloading the dataset.

In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import sys
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

#sys.path.insert(0, '../../')
# from edgeml.trainer.protoNNTrainer import ProtoNNTrainer
# from edgeml.graph.protoNN import ProtoNN
# import edgeml.utils as utils
# import helpermethods as helper
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


sys.path.append(r"D:\programming\practice\research\protoNN\EdgeML\examples\tf\ProtoNN")
import helpermethods as helper

Instructions for updating:
non-resource variables are not supported in the long term


# Helper Methods

In [2]:
#helper methods
sys.path.insert(0, '../')
import argparse


def getModelSize(matrixList, sparcityList, expected=True, bytesPerVar=4):
    '''
    expected: Expected size according to the parameters set. The number of
        zeros could actually be more than that is required to satisfy the
        sparsity constraint.
    '''
    nnzList, sizeList, isSparseList = [], [], []
    hasSparse = False
    for i in range(len(matrixList)):
        A, s = matrixList[i], sparcityList[i]
        assert A.ndim == 2
        assert s >= 0
        assert s <= 1
        nnz, size, sparse = countnnZ(A, s, bytesPerVar=bytesPerVar)
        nnzList.append(nnz)
        sizeList.append(size)
        hasSparse = (hasSparse or sparse)

    totalnnZ = np.sum(nnzList)
    totalSize = np.sum(sizeList)
    if expected:
        return totalnnZ, totalSize, hasSparse
    numNonZero = 0
    totalSize = 0
    hasSparse = False
    for i in range(len(matrixList)):
        A, s = matrixList[i], sparcityList[i]
        numNonZero_ = np.count_nonzero(A)
        numNonZero += numNonZero_
        hasSparse = (hasSparse or (s < 0.5))
        if s <= 0.5:
            totalSize += numNonZero_ * 2 * bytesPerVar
        else:
            totalSize += A.size * bytesPerVar
    return numNonZero, totalSize, hasSparse


def getGamma(gammaInit, projectionDim, dataDim, numPrototypes, x_train):
    if gammaInit is None:
        print("Using median heuristic to estimate gamma.")
        gamma, W, B = medianHeuristic(x_train, projectionDim,
                                            numPrototypes)
        print("Gamma estimate is: %f" % gamma)
        return W, B, gamma
    return None, None, gammaInit


def preprocessData(dataDir,w):
    '''
    Loads data from the dataDir and does some initial preprocessing
    steps. Data is assumed to be contained in two files,
    train.npy and test.npy. Each containing a 2D numpy array of dimension
    [numberOfExamples, numberOfFeatures + 1]. The first column of each
    matrix is assumed to contain label information.

    For an N-Class problem, we assume the labels are integers from 0 through
    N-1.
    '''
    # Uncomment for usual training data
    # train = np.load(dataDir + '/train_'+str(w)+'.npy')
    # test = np.load(dataDir + '/test_'+str(w)+'.npy')
    # Uncomment for time domain training data
    train = np.load(dataDir + '/ttrain_'+str(w)+'.npy')
    test = np.load(dataDir + '/ttest_'+str(w)+'.npy')
    # Uncomment for 1 sensordrop training data
    # train = np.load(dataDir + '/train_'+str(w)+'.npy')
    # test = np.load(dataDir + '/test_'+str(w)+'.npy')

    dataDimension = int(train.shape[1]) - 1
    x_train = train[:, 1:dataDimension + 1]
    y_train_ = train[:, 0]
    x_test = test[:, 1:dataDimension + 1]
    y_test_ = test[:, 0]

    numClasses = max(y_train_) - min(y_train_) + 1
    numClasses = max(numClasses, max(y_test_) - min(y_test_) + 1)
    numClasses = int(numClasses)

    # mean-var
    mean = np.mean(x_train, 0)
    std = np.std(x_train, 0)
    std[std[:] < 0.000001] = 1
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std

    # one hot y-train
    lab = y_train_.astype('uint8')
    lab = np.array(lab) - min(lab)
    lab_ = np.zeros((x_train.shape[0], numClasses))
    lab_[np.arange(x_train.shape[0]), lab] = 1
    y_train = lab_

    # one hot y-test
    lab = y_test_.astype('uint8')
    lab = np.array(lab) - min(lab)
    lab_ = np.zeros((x_test.shape[0], numClasses))
    lab_[np.arange(x_test.shape[0]), lab] = 1
    y_test = lab_

    return dataDimension, numClasses, x_train, y_train, x_test, y_test



def getProtoNNArgs():
    def checkIntPos(value):
        ivalue = int(value)
        if ivalue <= 0:
            raise argparse.ArgumentTypeError(
                "%s is an invalid positive int value" % value)
        return ivalue

    def checkIntNneg(value):
        ivalue = int(value)
        if ivalue < 0:
            raise argparse.ArgumentTypeError(
                "%s is an invalid non-neg int value" % value)
        return ivalue

    def checkFloatNneg(value):
        fvalue = float(value)
        if fvalue < 0:
            raise argparse.ArgumentTypeError(
                "%s is an invalid non-neg float value" % value)
        return fvalue

    def checkFloatPos(value):
        fvalue = float(value)
        if fvalue <= 0:
            raise argparse.ArgumentTypeError(
                "%s is an invalid positive float value" % value)
        return fvalue

    '''
    Parse protoNN commandline arguments
    '''
    parser = argparse.ArgumentParser(
        description='Hyperparameters for ProtoNN Algorithm')

    msg = 'Data directory containing train and test data. The '
    msg += 'data is assumed to be saved as 2-D numpy matrices with '
    msg += 'names `train.npy` and `test.npy`, of dimensions\n'
    msg += '\t[numberOfInstances, numberOfFeatures + 1].\n'
    msg += 'The first column of each file is assumed to contain label information.'
    msg += ' For a N-class problem, labels are assumed to be integers from 0 to'
    msg += ' N-1 (inclusive).'
    parser.add_argument('-d', '--data-dir', required=True, help=msg)
    parser.add_argument('-l', '--projection-dim', type=checkIntPos, default=10,
                        help='Projection Dimension.')
    parser.add_argument('-p', '--num-prototypes', type=checkIntPos, default=20,
                        help='Number of prototypes.')
    parser.add_argument('-g', '--gamma', type=checkFloatPos, default=None,
                        help='Gamma for Gaussian kernel. If not provided, ' +
                        'median heuristic will be used to estimate gamma.')

    parser.add_argument('-e', '--epochs', type=checkIntPos, default=100,
                        help='Total training epochs.')
    parser.add_argument('-b', '--batch-size', type=checkIntPos, default=32,
                        help='Batch size for each pass.')
    parser.add_argument('-r', '--learning-rate', type=checkFloatPos,
                        default=0.001,
                        help='Initial Learning rate for ADAM Optimizer.')

    parser.add_argument('-rW', type=float, default=0.000,
                        help='Coefficient for l2 regularizer for predictor' +
                        ' parameter W ' + '(default = 0.0).')
    parser.add_argument('-rB', type=float, default=0.00,
                        help='Coefficient for l2 regularizer for predictor' +
                        ' parameter B ' + '(default = 0.0).')
    parser.add_argument('-rZ', type=float, default=0.00,
                        help='Coefficient for l2 regularizer for predictor' +
                        'parameter Z ' +
                        '(default = 0.0).')

    parser.add_argument('-sW', type=float, default=1.000,
                        help='Sparsity constraint for predictor parameter W ' +
                        '(default = 1.0, i.e. dense matrix).')
    parser.add_argument('-sB', type=float, default=1.00,
                        help='Sparsity constraint for predictor parameter B ' +
                        '(default = 1.0, i.e. dense matrix).')
    parser.add_argument('-sZ', type=float, default=1.00,
                        help='Sparsity constraint for predictor parameter Z ' +
                        '(default = 1.0, i.e. dense matrix).')
    parser.add_argument('-pS', '--print-step', type=int, default=200,
                        help='The number of update steps between print ' +
                        'calls to console.')
    parser.add_argument('-vS', '--val-step', type=int, default=3,
                        help='The number of epochs between validation' +
                        'performance evaluation')
    return parser.parse_args()

# Utils 

In [3]:
#utils
import scipy.cluster
import scipy.spatial
import os


def medianHeuristic(data, projectionDimension, numPrototypes, W_init=None):
    '''
    This method can be used to estimate gamma for ProtoNN. An approximation to
    median heuristic is used here.
    1. First the data is collapsed into the projectionDimension by W_init. If
    W_init is not provided, it is initialized from a random normal(0, 1). Hence
    data normalization is essential.
    2. Prototype are computed by running a  k-means clustering on the projected
    data.
    3. The median distance is then estimated by calculating median distance
    between prototypes and projected data points.

    data needs to be [-1, numFeats]
    If using this method to initialize gamma, please use the W and B as well.

    TODO: Return estimate of Z (prototype labels) based on cluster centroids
    andand labels

    TODO: Clustering fails due to singularity error if projecting upwards

    W [dxd_cap]
    B [d_cap, m]
    returns gamma, W, B
    '''
    assert data.ndim == 2
    X = data
    featDim = data.shape[1]
    if projectionDimension > featDim:
        print("Warning: Projection dimension > feature dimension. Gamma")
        print("\t estimation due to median heuristic could fail.")
        print("\tTo retain the projection dataDimension, provide")
        print("\ta value for gamma.")

    if W_init is None:
        W_init = np.random.normal(size=[featDim, projectionDimension])
    W = W_init
    XW = np.matmul(X, W)
    assert XW.shape[1] == projectionDimension
    assert XW.shape[0] == len(X)
    # Requires [N x d_cap] data matrix of N observations of d_cap-dimension and
    # the number of centroids m. Returns, [n x d_cap] centroids and
    # elementwise center information.
    B, centers = scipy.cluster.vq.kmeans2(XW, numPrototypes)
    # Requires two matrices. Number of observations x dimension of observation
    # space. Distances[i,j] is the distance between XW[i] and B[j]
    distances = scipy.spatial.distance.cdist(XW, B, metric='euclidean')
    distances = np.reshape(distances, [-1])
    gamma = np.median(distances)
    gamma = 1 / (2.5 * gamma)
    return gamma.astype('float32'), W.astype('float32'), B.T.astype('float32')


def multiClassHingeLoss(logits, label, batch_th):
    '''
    MultiClassHingeLoss to match C++ Version - No TF internal version
    '''
    flatLogits = tf.reshape(logits, [-1, ])
    label_ = tf.argmax(label, 1)

    correctId = tf.range(0, batch_th) * label.shape[1] + label_
    correctLogit = tf.gather(flatLogits, correctId)

    maxLabel = tf.argmax(logits, 1)
    top2, _ = tf.nn.top_k(logits, k=2, sorted=True)

    wrongMaxLogit = tf.where(
        tf.equal(maxLabel, label_), top2[:, 1], top2[:, 0])

    return tf.reduce_mean(tf.nn.relu(1. + wrongMaxLogit - correctLogit))


def crossEntropyLoss(logits, label):
    '''
    Cross Entropy loss for MultiClass case in joint training for
    faster convergence
    '''
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                   labels=tf.stop_gradient(label)))


def mean_absolute_error(logits, label):
    '''
    Function to compute the mean absolute error.
    '''
    return tf.reduce_mean(tf.abs(tf.subtract(logits, label)))


def hardThreshold(A, s):
    '''
    Hard thresholding function on Tensor A with sparsity s
    '''
    A_ = np.copy(A)
    A_ = A_.ravel()
    if len(A_) > 0:
        th = np.percentile(np.abs(A_), (1 - s) * 100.0, interpolation='higher')
        A_[np.abs(A_) < th] = 0.0
    A_ = A_.reshape(A.shape)
    return A_


def copySupport(src, dest):
    '''
    copy support of src tensor to dest tensor
    '''
    support = np.nonzero(src)
    dest_ = dest
    dest = np.zeros(dest_.shape)
    dest[support] = dest_[support]
    return dest


def countnnZ(A, s, bytesPerVar=4):
    '''
    Returns # of non-zeros and representative size of the tensor
    Uses dense for s >= 0.5 - 4 byte
    Else uses sparse - 8 byte
    '''
    params = 1
    hasSparse = False
    for i in range(0, len(A.shape)):
        params *= int(A.shape[i])
    if s < 0.5:
        nnZ = np.ceil(params * s)
        hasSparse = True
        return nnZ, nnZ * 2 * bytesPerVar, hasSparse
    else:
        nnZ = params
        return nnZ, nnZ * bytesPerVar, hasSparse


def getConfusionMatrix(predicted, target, numClasses):
    '''
    Returns a confusion matrix for a multiclass classification
    problem. `predicted` is a 1-D array of integers representing
    the predicted classes and `target` is the target classes.

    confusion[i][j]: Number of elements of class j
        predicted as class i
    Labels are assumed to be in range(0, numClasses)
    Use`printFormattedConfusionMatrix` to echo the confusion matrix
    in a user friendly form.
    '''
    assert(predicted.ndim == 1)
    assert(target.ndim == 1)
    arr = np.zeros([numClasses, numClasses])

    for i in range(len(predicted)):
        arr[predicted[i]][target[i]] += 1
    return arr


def printFormattedConfusionMatrix(matrix):
    '''
    Given a 2D confusion matrix, prints it in a human readable way.
    The confusion matrix is expected to be a 2D numpy array with
    square dimensions
    '''
    assert(matrix.ndim == 2)
    assert(matrix.shape[0] == matrix.shape[1])
    RECALL = 'Recall'
    PRECISION = 'PRECISION'
    print("|%s|" % ('True->'), end='')
    for i in range(matrix.shape[0]):
        print("%7d|" % i, end='')
    print("%s|" % 'Precision')

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))

    precisionlist = np.sum(matrix, axis=1)
    recalllist = np.sum(matrix, axis=0)
    precisionlist = [matrix[i][i] / x if x !=
                     0 else -1 for i, x in enumerate(precisionlist)]
    recalllist = [matrix[i][i] / x if x !=
                  0 else -1 for i, x in enumerate(recalllist)]
    for i in range(matrix.shape[0]):
        # len recall = 6
        print("|%6d|" % (i), end='')
        for j in range(matrix.shape[0]):
            print("%7d|" % (matrix[i][j]), end='')
        print("%s" % (" " * (len(PRECISION) - 7)), end='')
        if precisionlist[i] != -1:
            print("%1.5f|" % precisionlist[i])
        else:
            print("%7s|" % "nan")

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))
    print("|%s|" % ('Recall'), end='')

    for i in range(matrix.shape[0]):
        if recalllist[i] != -1:
            print("%1.5f|" % (recalllist[i]), end='')
        else:
            print("%7s|" % "nan", end='')

    print('%s|' % (' ' * len(PRECISION)))


def getPrecisionRecall(cmatrix, label=1):
    trueP = cmatrix[label][label]
    denom = np.sum(cmatrix, axis=0)[label]
    if denom == 0:
        denom = 1
    recall = trueP / denom
    denom = np.sum(cmatrix, axis=1)[label]
    if denom == 0:
        denom = 1
    precision = trueP / denom
    return precision, recall


def getMacroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    precision = np.sum(precisionlist__)
    precision /= len(precisionlist__)
    recall = np.sum(recalllist__)
    recall /= len(recalllist__)
    return precision, recall


def getMicroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    num = 0.0
    for i in range(len(cmatrix)):
        num += cmatrix[i][i]

    precision = num / np.sum(precisionlist)
    recall = num / np.sum(recalllist)
    return precision, recall


def getMacroMicroFScore(cmatrix):
    '''
    Returns macro and micro f-scores.
    Refer: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.8244&rep=rep1&type=pdf
    '''
    precisionlist = np.sum(cmatrix, axis=1)
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    macro = 0.0
    for i in range(len(precisionlist)):
        denom = precisionlist__[i] + recalllist__[i]
        numer = precisionlist__[i] * recalllist__[i] * 2
        if denom == 0:
            denom = 1
        macro += numer / denom
    macro /= len(precisionlist)

    num = 0.0
    for i in range(len(precisionlist)):
        num += cmatrix[i][i]

    denom1 = np.sum(precisionlist)
    denom2 = np.sum(recalllist)
    pi = num / denom1
    rho = num / denom2
    denom = pi + rho
    if denom == 0:
        denom = 1
    micro = 2 * pi * rho / denom
    return macro, micro


class GraphManager:
    '''
    Manages saving and restoring graphs. Designed to be used with EMI-RNN
    though is general enough to be useful otherwise as well.
    '''

    def __init__(self):
        pass

    def checkpointModel(self, saver, sess, modelPrefix,
                        globalStep=1000, redirFile=None):
        saver.save(sess, modelPrefix, global_step=globalStep)
        print('Model saved to %s, global_step %d' % (modelPrefix, globalStep),
              file=redirFile)

    def loadCheckpoint(self, sess, modelPrefix, globalStep,
                       redirFile=None):
        metaname = modelPrefix + '-%d.meta' % globalStep
        basename = os.path.basename(metaname)
        fileList = os.listdir(os.path.dirname(modelPrefix))
        fileList = [x for x in fileList if x.startswith(basename)]
        assert len(fileList) > 0, 'Checkpoint file not found'
        msg = 'Too many or too few checkpoint files for globalStep: %d' % globalStep
        assert len(fileList) is 1, msg
        chkpt = basename + '/' + fileList[0]
        saver = tf.train.import_meta_graph(metaname)
        metaname = metaname[:-5]
        saver.restore(sess, metaname)
        graph = tf.get_default_graph()
        return graph

# Model Trainer - ProtoNN

In [4]:
#Trainer
class ProtoNNTrainer:
    def __init__(self, protoNNObj, regW, regB, regZ,
                 sparcityW, sparcityB, sparcityZ,
                 learningRate, X, Y, lossType='l2'):
        '''
        A wrapper for the various techniques used for training ProtoNN. This
        subsumes both the responsibility of loss graph construction and
        performing training. The original training routine that is part of the
        C++ implementation of EdgeML used iterative hard thresholding (IHT),
        gamma estimation through median heuristic and other tricks for
        training ProtoNN. This module implements the same in Tensorflow
        and python.

        protoNNObj: An instance of ProtoNN class defining the forward
            computation graph. The loss functions and training routines will be
            attached to this instance.
        regW, regB, regZ: Regularization constants for W, B, and
            Z matrices of protoNN.
        sparcityW, sparcityB, sparcityZ: Sparsity constraints
            for W, B and Z matrices. A value between 0 (exclusive) and 1
            (inclusive) is expected. A value of 1 indicates dense training.
        learningRate: Initial learning rate for ADAM optimizer.
        X, Y : Placeholders for data and labels.
            X [-1, featureDimension]
            Y [-1, num Labels]
        lossType: ['l2', 'xentropy']
        '''
        self.protoNNObj = protoNNObj
        self.__regW = regW
        self.__regB = regB
        self.__regZ = regZ
        self.__sW = sparcityW
        self.__sB = sparcityB
        self.__sZ = sparcityZ
        self.__lR = learningRate
        self.X = X
        self.Y = Y
        self.sparseTraining = True
        if (sparcityW == 1.0) and (sparcityB == 1.0) and (sparcityZ == 1.0):
            self.sparseTraining = False
            print("Sparse training disabled.", file=sys.stderr)
        # Define placeholders for sparse training
        self.W_th = None
        self.B_th = None
        self.Z_th = None
        self.__lossType = lossType
        self.__validInit = False
        self.__validInit = self.__validateInit()
        self.__protoNNOut = protoNNObj(X, Y)
        self.loss = self.__lossGraph()
        self.trainStep = self.__trainGraph()
        self.__hthOp = self.__getHardThresholdOp()
        self.accuracy = protoNNObj.getAccuracyOp()

    def __validateInit(self):
        self.__validInit = False
        msg = "Sparsity value should be between"
        msg += " 0 and 1 (both inclusive)."
        assert self.__sW >= 0. and self.__sW <= 1., 'W:' + msg
        assert self.__sB >= 0. and self.__sB <= 1., 'B:' + msg
        assert self.__sZ >= 0. and self.__sZ <= 1., 'Z:' + msg
        d, dcap, m, L, _ = self.protoNNObj.getHyperParams()
        msg = 'Y should be of dimension [-1, num labels/classes]'
        msg += ' specified as part of ProtoNN object.'
        assert (len(self.Y.shape)) == 2, msg
        assert (self.Y.shape[1] == L), msg
        msg = 'X should be of dimension [-1, featureDimension]'
        msg += ' specified as part of ProtoNN object.'
        assert (len(self.X.shape) == 2), msg
        assert (self.X.shape[1] == d), msg
        self.__validInit = True
        msg = 'Values can be \'l2\', or \'xentropy\''
        if self.__lossType not in ['l2', 'xentropy']:
            raise ValueError(msg)
        return True

    def __lossGraph(self):
        pnnOut = self.__protoNNOut
        l1, l2, l3 = self.__regW, self.__regB, self.__regZ
        W, B, Z, _ = self.protoNNObj.getModelMatrices()
        if self.__lossType == 'l2':
            with tf.name_scope('protonn-l2-loss'):
                loss_0 = tf.nn.l2_loss(self.Y - pnnOut)
                reg = l1 * tf.nn.l2_loss(W) + l2 * tf.nn.l2_loss(B)
                reg += l3 * tf.nn.l2_loss(Z)
                loss = loss_0 + reg
        elif self.__lossType == 'xentropy':
            with tf.name_scope('protonn-xentropy-loss'):
                loss_0 = tf.nn.softmax_cross_entropy_with_logits_v2(logits=pnnOut,
                                                         labels=tf.stop_gradient(self.Y))
                loss_0 = tf.reduce_mean(loss_0)
                reg = l1 * tf.nn.l2_loss(W) + l2 * tf.nn.l2_loss(B)
                reg += l3 * tf.nn.l2_loss(Z)
                loss = loss_0 + reg
        return loss

    def __trainGraph(self):
        with tf.name_scope('protonn-gradient-adam'):
            trainStep = tf.train.AdamOptimizer(self.__lR)
            trainStep = trainStep.minimize(self.loss)
        return trainStep

    def __getHardThresholdOp(self):
        W, B, Z, _ = self.protoNNObj.getModelMatrices()
        self.W_th = tf.placeholder(tf.float32, name='W_th')
        self.B_th = tf.placeholder(tf.float32, name='B_th')
        self.Z_th = tf.placeholder(tf.float32, name='Z_th')
        with tf.name_scope('hard-threshold-assignments'):
            hard_thrsd_W = W.assign(self.W_th)
            hard_thrsd_B = B.assign(self.B_th)
            hard_thrsd_Z = Z.assign(self.Z_th)
            hard_thrsd_op = tf.group(hard_thrsd_W, hard_thrsd_B, hard_thrsd_Z)
        return hard_thrsd_op

    def train(self, batchSize, totalEpochs, sess,
              x_train, x_val, y_train, y_val, noInit=False,
              redirFile=None, printStep=10, valStep=3):
        '''
        Performs dense training of ProtoNN followed by iterative hard
        thresholding to enforce sparsity constraints.

        batchSize: Batch size per update
        totalEpochs: The number of epochs to run training for. One epoch is
            defined as one pass over the entire training data.
        sess: The Tensorflow session to use for running various graph
            operators.
        x_train, x_val, y_train, y_val: The numpy array containing train and
            validation data. x data is assumed to in of shape [-1,
            featureDimension] while y should have shape [-1, numberLabels].
        noInit: By default, all the tensors of the computation graph are
        initialized at the start of the training session. Set noInit=False to
        disable this behaviour.
        printStep: Number of batches between echoing of loss and train accuracy.
        valStep: Number of epochs between evolutions on validation set.
        '''
        d, d_cap, m, L, gamma = self.protoNNObj.getHyperParams()
        assert batchSize >= 1, 'Batch size should be positive integer'
        assert totalEpochs >= 1, 'Total epochs should be positive integer'
        assert x_train.ndim == 2, 'Expected training data to be of rank 2'
        assert x_train.shape[1] == d, 'Expected x_train to be [-1, %d]' % d
        assert x_val.ndim == 2, 'Expected validation data to be of rank 2'
        assert x_val.shape[1] == d, 'Expected x_val to be [-1, %d]' % d
        assert y_train.ndim == 2, 'Expected training labels to be of rank 2'
        assert y_train.shape[1] == L, 'Expected y_train to be [-1, %d]' % L
        assert y_val.ndim == 2, 'Expected validation labels to be of rank 2'
        assert y_val.shape[1] == L, 'Expected y_val to be [-1, %d]' % L

        # Numpy will throw asserts for arrays
        if sess is None:
            raise ValueError('sess must be valid Tensorflow session.')

        trainNumBatches = int(np.ceil(len(x_train) / batchSize))
        valNumBatches = int(np.ceil(len(x_val) / batchSize))
        x_train_batches = np.array_split(x_train, trainNumBatches)
        y_train_batches = np.array_split(y_train, trainNumBatches)
        x_val_batches = np.array_split(x_val, valNumBatches)
        y_val_batches = np.array_split(y_val, valNumBatches)
        if not noInit:
            sess.run(tf.global_variables_initializer())
        X, Y = self.X, self.Y
        W, B, Z, _ = self.protoNNObj.getModelMatrices()
        for epoch in range(totalEpochs):
            for i in range(len(x_train_batches)):
                batch_x = x_train_batches[i]
                batch_y = y_train_batches[i]
                feed_dict = {
                    X: batch_x,
                    Y: batch_y
                }
                sess.run(self.trainStep, feed_dict=feed_dict)
                if i % printStep == 0:
                    loss, acc = sess.run([self.loss, self.accuracy],
                                         feed_dict=feed_dict)
                    msg = "Epoch: %3d Batch: %3d" % (epoch, i)
                    msg += " Loss: %3.5f Accuracy: %2.5f" % (loss, acc)
                    print(msg, file=redirFile)

            # Perform Hard thresholding
            if self.sparseTraining:
                W_, B_, Z_ = sess.run([W, B, Z])
                fd_thrsd = {
                    self.W_th: hardThreshold(W_, self.__sW),
                    self.B_th: hardThreshold(B_, self.__sB),
                    self.Z_th: hardThreshold(Z_, self.__sZ)
                }
                sess.run(self.__hthOp, feed_dict=fd_thrsd)

            if (epoch + 1) % valStep  == 0:
                acc = 0.0
                loss = 0.0
                for j in range(len(x_val_batches)):
                    batch_x = x_val_batches[j]
                    batch_y = y_val_batches[j]
                    feed_dict = {
                        X: batch_x,
                        Y: batch_y
                    }
                    acc_, loss_ = sess.run([self.accuracy, self.loss],
                                           feed_dict=feed_dict)
                    acc += acc_
                    loss += loss_
                acc /= len(y_val_batches)
                loss /= len(y_val_batches)
                print("Test Loss: %2.5f Accuracy: %2.5f" % (loss, acc))


# Model Graph - ProtoNN

In [5]:

class ProtoNN:
    def __init__(self, inputDimension, projectionDimension, numPrototypes,
                 numOutputLabels, gamma,
                 W = None, B = None, Z = None):
        '''
        Forward computation graph for ProtoNN.

        inputDimension: Input data dimension or feature dimension.
        projectionDimension: hyperparameter
        numPrototypes: hyperparameter
        numOutputLabels: The number of output labels or classes
        W, B, Z: Numpy matrices that can be used to initialize
            projection matrix(W), prototype matrix (B) and prototype labels
            matrix (B).
            Expected Dimensions:
                W   inputDimension (d) x projectionDimension (d_cap)
                B   projectionDimension (d_cap) x numPrototypes (m)
                Z   numOutputLabels (L) x numPrototypes (m)
        '''
        with tf.name_scope('protoNN') as ns:
            self.__nscope = ns
        self.__d = inputDimension
        self.__d_cap = projectionDimension
        self.__m = numPrototypes
        self.__L = numOutputLabels

        self.__inW = W
        self.__inB = B
        self.__inZ = Z
        self.__inGamma = gamma
        self.W, self.B, self.Z = None, None, None
        self.gamma = None

        self.__validInit = False
        self.__initWBZ()
        self.__initGamma()
        self.__validateInit()
        self.protoNNOut = None
        self.predictions = None
        self.accuracy = None

    def __validateInit(self):
        self.__validInit = False
        errmsg = "Dimensions mismatch! Should be W[d, d_cap]"
        errmsg += ", B[d_cap, m] and Z[L, m]"
        d, d_cap, m, L, _ = self.getHyperParams()
        assert self.W.shape[0] == d, errmsg
        assert self.W.shape[1] == d_cap, errmsg
        assert self.B.shape[0] == d_cap, errmsg
        assert self.B.shape[1] == m, errmsg
        assert self.Z.shape[0] == L, errmsg
        assert self.Z.shape[1] == m, errmsg
        self.__validInit = True

    def __initWBZ(self):
        with tf.name_scope(self.__nscope):
            W = self.__inW
            if W is None:
                W = tf.random_normal_initializer()
                W = W([self.__d, self.__d_cap])
            self.W = tf.Variable(W, name='W', dtype=tf.float32)

            B = self.__inB
            if B is None:
                B = tf.random_uniform_initializer()
                B = B([self.__d_cap, self.__m])
            self.B = tf.Variable(B, name='B', dtype=tf.float32)

            Z = self.__inZ
            if Z is None:
                Z = tf.random_normal_initializer()
                Z = Z([self.__L, self.__m])
            Z = tf.Variable(Z, name='Z', dtype=tf.float32)
            self.Z = Z
        return self.W, self.B, self.Z

    def __initGamma(self):
        with tf.name_scope(self.__nscope):
            gamma = self.__inGamma
            self.gamma = tf.constant(gamma, name='gamma')

    def getHyperParams(self):
        '''
        Returns the model hyperparameters:
            [inputDimension, projectionDimension,
            numPrototypes, numOutputLabels, gamma]
        '''
        d = self.__d
        dcap = self.__d_cap
        m = self.__m
        L = self.__L
        return d, dcap, m, L, self.gamma

    def getModelMatrices(self):
        '''
        Returns Tensorflow tensors of the model matrices, which
        can then be evaluated to obtain corresponding numpy arrays.

        These can then be exported as part of other implementations of
        ProtonNN, for instance a C++ implementation or pure python
        implementation.
        Returns
            [ProjectionMatrix (W), prototypeMatrix (B),
             prototypeLabelsMatrix (Z), gamma]
        '''
        return self.W, self.B, self.Z, self.gamma

    def __call__(self, X, Y=None):
        '''
        This method is responsible for construction of the forward computation
        graph. The end point of the computation graph, or in other words the
        output operator for the forward computation is returned. Additionally,
        if the argument Y is provided, a classification accuracy operator with
        Y as target will also be created. For this, Y is assumed to in one-hot
        encoded format and the class with the maximum prediction score is
        compared to the encoded class in Y.  This accuracy operator is returned
        by getAccuracyOp() method. If a different accuracyOp is required, it
        can be defined by overriding the createAccOp(protoNNScoresOut, Y)
        method.

        X: Input tensor or placeholder of shape [-1, inputDimension]
        Y: Optional tensor or placeholder for targets (labels or classes).
            Expected shape is [-1, numOutputLabels].
        returns: The forward computation outputs, self.protoNNOut
        '''
        # This should never execute
        assert self.__validInit is True, "Initialization failed!"
        if self.protoNNOut is not None:
            return self.protoNNOut

        W, B, Z, gamma = self.W, self.B, self.Z, self.gamma
        with tf.name_scope(self.__nscope):
            WX = tf.matmul(X, W)
            # Convert WX to tensor so that broadcasting can work
            dim = [-1, WX.shape.as_list()[1], 1]
            WX = tf.reshape(WX, dim)
            dim = [1, B.shape.as_list()[0], -1]
            B = tf.reshape(B, dim)
            l2sim = B - WX
            l2sim = tf.pow(l2sim, 2)
            l2sim = tf.reduce_sum(l2sim, 1, keepdims=True)
            self.l2sim = l2sim
            gammal2sim = (-1 * gamma * gamma) * l2sim
            M = tf.exp(gammal2sim)
            dim = [1] + Z.shape.as_list()
            Z = tf.reshape(Z, dim)
            y = tf.multiply(Z, M)
            y = tf.reduce_sum(y, 2, name='protoNNScoreOut')
            self.protoNNOut = y
            self.predictions = tf.argmax(y, 1, name='protoNNPredictions')
            if Y is not None:
                self.createAccOp(self.protoNNOut, Y)
        return y

    def createAccOp(self, outputs, target):
        '''
        Define an accuracy operation on ProtoNN's output scores and targets.
        Here a simple classification accuracy operator is defined. More
        complicated operators (for multiple label problems and so forth) can be
        defined by overriding this method
        '''
        assert self.predictions is not None
        target = tf.argmax(target, 1)
        correctPrediction = tf.equal(self.predictions, target)
        acc = tf.reduce_mean(tf.cast(correctPrediction, tf.float32),
                             name='protoNNAccuracy')
        self.accuracy = acc

    def getPredictionsOp(self):
        '''
        The predictions operator is defined as argmax(protoNNScores) for each
        prediction.
        '''
        return self.predictions

    def getAccuracyOp(self):
        '''
        returns accuracyOp as defined by createAccOp. It defaults to
        multi-class classification accuracy.
        '''
        msg = "Accuracy operator not defined in graph. Did you provide Y as an"
        msg += " argument to _call_?"
        assert self.accuracy is not None, msg
        return self.accuracy

# Obtain Data

It is assumed that the Daphnet data has already been downloaded,preprocessed and set up in subdirectory.

In [6]:
DATA_DIR = r"./experiments"
windowLen = 'data'
out = preprocessData(DATA_DIR,windowLen)
dataDimension = out[0]
numClasses = out[1]
x_train, y_train = out[2], out[3]
x_test, y_test = out[4], out[5]
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  423
Num classes:  2


In [7]:
DATA_DIR = r"./experiments"
train, test = np.load(DATA_DIR + '/ttrain_data.npy'), np.load(DATA_DIR + '/ttest_data.npy')
x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]

numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 1)
numClasses = int(numClasses)

y_train = helper.to_onehot(y_train, numClasses)
y_test = helper.to_onehot(y_test, numClasses)
dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]

# Model Parameters

Note that ProtoNN is very sensitive to the value of the hyperparameter $\gamma$, here stored in valiable `GAMMA`. If `GAMMA` is set to `None`, median heuristic will be used to estimate a good value of $\gamma$ through the `helper.getGamma()` method. This method also returns the corresponding `W` and `B` matrices which should be used to initialize ProtoNN (as is done here).

In [8]:
PROJECTION_DIM = 5 #d^
NUM_PROTOTYPES = 40 #m
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 1.0
SPAR_B = 0.8
SPAR_Z = 0.8
LEARNING_RATE = 0.001
NUM_EPOCHS = 600
BATCH_SIZE = 32
GAMMA = 0.007586

In [9]:
W, B, gamma = getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [10]:
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,classification_report
from functools import partial

X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
def objective(trial,x_train, x_test, y_train, y_test):
    W, B, gamma = getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)
    # Inside the optimization function, you use the 'trial' object to suggest hyperparameters
    REG_W = trial.suggest_float('REG_W', 2e-6, 5e-6)
    REG_B = trial.suggest_float('REG_B', 0.0, 0.01)
    REG_Z = trial.suggest_float('REG_Z', 2e-5, 5e-5)
    SPAR_W = trial.suggest_float('SPAR_W', 0.5, 1.0)
    SPAR_B = trial.suggest_float('SPAR_B', 0.5, 1.0)
    SPAR_Z = trial.suggest_float('SPAR_Z', 0.5, 1.0)
    LEARNING_RATE = trial.suggest_float('LEARNING_RATE', 1e-4, 1e-3)
    NUM_EPOCHS = trial.suggest_int('NUM_EPOCHS', 200, 600)

    # Set the suggested hyperparameters in the trainer
    protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
    trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
    # Call your ProtoNN trainer function or use it as needed
    sess = tf.Session()

    trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,printStep=600, valStep=10)
    acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
    pred = sess.run(protoNN.predictions, feed_dict={X: x_test, Y: y_test})
    # W, B, Z are tensorflow graph nodes
    W, B, Z, _ = protoNN.getModelMatrices()
    matrixList = sess.run([W, B, Z])
    sparcityList = [SPAR_W, SPAR_B, SPAR_Z]                       
    nnz, size, sparse = getModelSize(matrixList, sparcityList)
    y_test = np.argmax(y_test,axis=1)
    sensitivity = confusion_matrix(y_test,pred)[1][1]/(confusion_matrix(y_test,pred)[1][1] + confusion_matrix(y_test,pred)[1][0])
    specificity = confusion_matrix(y_test,pred)[0][0]/(confusion_matrix(y_test,pred)[0][0] + confusion_matrix(y_test,pred)[0][1])
    return (sensitivity+specificity)/2



In [12]:
import optuna
study = optuna.create_study(direction='maximize')


[I 2023-11-17 13:07:07,306] A new study created in memory with name: no-name-f0231465-89ff-4f90-a3ca-24098684743e


In [13]:
op_fun = partial(objective,x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)
study.optimize(op_fun,n_trials=20)

Epoch:   0 Batch:   0 Loss: 0.28062 Accuracy: 1.00000
Epoch:   1 Batch:   0 Loss: 0.17970 Accuracy: 1.00000
Epoch:   2 Batch:   0 Loss: 0.15184 Accuracy: 1.00000
Epoch:   3 Batch:   0 Loss: 0.37073 Accuracy: 1.00000
Epoch:   4 Batch:   0 Loss: 0.87558 Accuracy: 0.00000
Epoch:   5 Batch:   0 Loss: 1.34357 Accuracy: 0.00000
Epoch:   6 Batch:   0 Loss: 1.64776 Accuracy: 0.00000
Epoch:   7 Batch:   0 Loss: 1.81831 Accuracy: 0.00000
Epoch:   8 Batch:   0 Loss: 1.90185 Accuracy: 0.00000
Epoch:   9 Batch:   0 Loss: 1.93356 Accuracy: 0.00000
Test Loss: 1.09768 Accuracy: 0.50020
Epoch:  10 Batch:   0 Loss: 1.93526 Accuracy: 0.00000
Epoch:  11 Batch:   0 Loss: 1.92006 Accuracy: 0.00000
Epoch:  12 Batch:   0 Loss: 1.89471 Accuracy: 0.00000
Epoch:  13 Batch:   0 Loss: 1.86478 Accuracy: 0.00000
Epoch:  14 Batch:   0 Loss: 1.83123 Accuracy: 0.00000
Epoch:  15 Batch:   0 Loss: 1.79649 Accuracy: 0.00000
Epoch:  16 Batch:   0 Loss: 1.76071 Accuracy: 0.00000
Epoch:  17 Batch:   0 Loss: 1.72452 Accuracy:

Epoch: 143 Batch:   0 Loss: 0.77258 Accuracy: 0.00000
Epoch: 144 Batch:   0 Loss: 0.77259 Accuracy: 0.00000
Epoch: 145 Batch:   0 Loss: 0.77261 Accuracy: 0.00000
Epoch: 146 Batch:   0 Loss: 0.77264 Accuracy: 0.00000
Epoch: 147 Batch:   0 Loss: 0.77268 Accuracy: 0.00000
Epoch: 148 Batch:   0 Loss: 0.77272 Accuracy: 0.00000
Epoch: 149 Batch:   0 Loss: 0.77278 Accuracy: 0.00000
Test Loss: 0.70668 Accuracy: 0.50070
Epoch: 150 Batch:   0 Loss: 0.77285 Accuracy: 0.00000
Epoch: 151 Batch:   0 Loss: 0.77291 Accuracy: 0.00000
Epoch: 152 Batch:   0 Loss: 0.77298 Accuracy: 0.00000
Epoch: 153 Batch:   0 Loss: 0.77307 Accuracy: 0.00000
Epoch: 154 Batch:   0 Loss: 0.77315 Accuracy: 0.00000
Epoch: 155 Batch:   0 Loss: 0.77323 Accuracy: 0.00000
Epoch: 156 Batch:   0 Loss: 0.77332 Accuracy: 0.00000
Epoch: 157 Batch:   0 Loss: 0.77342 Accuracy: 0.00000
Epoch: 158 Batch:   0 Loss: 0.77352 Accuracy: 0.00000
Epoch: 159 Batch:   0 Loss: 0.77362 Accuracy: 0.00000
Test Loss: 0.70282 Accuracy: 0.50070
Epoch: 1

[I 2023-11-17 13:09:24,795] Trial 0 finished with value: 0.5004020908725372 and parameters: {'REG_W': 3.810381964194984e-06, 'REG_B': 0.008502786220215441, 'REG_Z': 3.65253497805578e-05, 'SPAR_W': 0.8805874465018864, 'SPAR_B': 0.6587546957396175, 'SPAR_Z': 0.5053907565086231, 'LEARNING_RATE': 0.00040739436645659944, 'NUM_EPOCHS': 260}. Best is trial 0 with value: 0.5004020908725372.


Test Loss: 0.68225 Accuracy: 0.50060
Epoch:   0 Batch:   0 Loss: 1.30456 Accuracy: 0.00000
Epoch:   1 Batch:   0 Loss: 4.70260 Accuracy: 0.00000
Epoch:   2 Batch:   0 Loss: 3.37755 Accuracy: 0.00000
Epoch:   3 Batch:   0 Loss: 3.04379 Accuracy: 0.00000
Epoch:   4 Batch:   0 Loss: 2.93987 Accuracy: 0.00000
Epoch:   5 Batch:   0 Loss: 2.88502 Accuracy: 0.00000
Epoch:   6 Batch:   0 Loss: 2.83778 Accuracy: 0.00000
Epoch:   7 Batch:   0 Loss: 2.78877 Accuracy: 0.00000
Epoch:   8 Batch:   0 Loss: 2.73576 Accuracy: 0.00000
Epoch:   9 Batch:   0 Loss: 2.67984 Accuracy: 0.00000
Test Loss: 1.41102 Accuracy: 0.50020
Epoch:  10 Batch:   0 Loss: 2.62035 Accuracy: 0.00000
Epoch:  11 Batch:   0 Loss: 2.55769 Accuracy: 0.00000
Epoch:  12 Batch:   0 Loss: 2.49231 Accuracy: 0.00000
Epoch:  13 Batch:   0 Loss: 2.42448 Accuracy: 0.00000
Epoch:  14 Batch:   0 Loss: 2.35303 Accuracy: 0.00000
Epoch:  15 Batch:   0 Loss: 2.28109 Accuracy: 0.00000
Epoch:  16 Batch:   0 Loss: 2.20536 Accuracy: 0.00000
Epoch:  

Epoch: 142 Batch:   0 Loss: 0.75975 Accuracy: 0.00000
Epoch: 143 Batch:   0 Loss: 0.75937 Accuracy: 0.00000
Epoch: 144 Batch:   0 Loss: 0.75900 Accuracy: 0.00000
Epoch: 145 Batch:   0 Loss: 0.75866 Accuracy: 0.00000
Epoch: 146 Batch:   0 Loss: 0.75833 Accuracy: 0.00000
Epoch: 147 Batch:   0 Loss: 0.75801 Accuracy: 0.00000
Epoch: 148 Batch:   0 Loss: 0.75771 Accuracy: 0.00000
Epoch: 149 Batch:   0 Loss: 0.75743 Accuracy: 0.00000
Test Loss: 0.69773 Accuracy: 0.50060
Epoch: 150 Batch:   0 Loss: 0.75716 Accuracy: 0.00000
Epoch: 151 Batch:   0 Loss: 0.75690 Accuracy: 0.00000
Epoch: 152 Batch:   0 Loss: 0.75665 Accuracy: 0.00000
Epoch: 153 Batch:   0 Loss: 0.75640 Accuracy: 0.00000
Epoch: 154 Batch:   0 Loss: 0.75616 Accuracy: 0.00000
Epoch: 155 Batch:   0 Loss: 0.75594 Accuracy: 0.00000
Epoch: 156 Batch:   0 Loss: 0.75572 Accuracy: 0.00000
Epoch: 157 Batch:   0 Loss: 0.75552 Accuracy: 0.00000
Epoch: 158 Batch:   0 Loss: 0.75531 Accuracy: 0.00000
Epoch: 159 Batch:   0 Loss: 0.75512 Accuracy:

...

[I 2023-11-17 14:46:03,644] Trial 18 finished with value: 0.5008041817450744 and parameters: {'REG_W': 2.7986508078728687e-06, 'REG_B': 0.0016520481634988342, 'REG_Z': 4.701659115784846e-05, 'SPAR_W': 0.9952675904405462, 'SPAR_B': 0.899331900603328, 'SPAR_Z': 0.8610259893007911, 'LEARNING_RATE': 0.0005469267034550651, 'NUM_EPOCHS': 505}. Best is trial 10 with value: 0.7833735424205871.


Epoch:   0 Batch:   0 Loss: 0.05067 Accuracy: 1.00000
Epoch:   1 Batch:   0 Loss: 0.04223 Accuracy: 1.00000
Epoch:   2 Batch:   0 Loss: 0.36415 Accuracy: 1.00000
Epoch:   3 Batch:   0 Loss: 1.36094 Accuracy: 0.00000
Epoch:   4 Batch:   0 Loss: 1.59576 Accuracy: 0.00000
Epoch:   5 Batch:   0 Loss: 1.64890 Accuracy: 0.00000
Epoch:   6 Batch:   0 Loss: 1.67048 Accuracy: 0.00000
Epoch:   7 Batch:   0 Loss: 1.67943 Accuracy: 0.00000
Epoch:   8 Batch:   0 Loss: 1.68064 Accuracy: 0.00000
Epoch:   9 Batch:   0 Loss: 1.67594 Accuracy: 0.00000
Test Loss: 0.95305 Accuracy: 0.50020
Epoch:  10 Batch:   0 Loss: 1.66648 Accuracy: 0.00000
Epoch:  11 Batch:   0 Loss: 1.65380 Accuracy: 0.00000
Epoch:  12 Batch:   0 Loss: 1.63879 Accuracy: 0.00000
Epoch:  13 Batch:   0 Loss: 1.62171 Accuracy: 0.00000
Epoch:  14 Batch:   0 Loss: 1.60405 Accuracy: 0.00000
Epoch:  15 Batch:   0 Loss: 1.58575 Accuracy: 0.00000
Epoch:  16 Batch:   0 Loss: 1.56746 Accuracy: 0.00000
Epoch:  17 Batch:   0 Loss: 1.54984 Accuracy:

Epoch: 143 Batch:   0 Loss: 0.85687 Accuracy: 0.00000
Epoch: 144 Batch:   0 Loss: 0.85624 Accuracy: 0.00000
Epoch: 145 Batch:   0 Loss: 0.85563 Accuracy: 0.00000
Epoch: 146 Batch:   0 Loss: 0.85503 Accuracy: 0.00000
Epoch: 147 Batch:   0 Loss: 0.85443 Accuracy: 0.00000
Epoch: 148 Batch:   0 Loss: 0.85386 Accuracy: 0.00000
Epoch: 149 Batch:   0 Loss: 0.85329 Accuracy: 0.00000
Test Loss: 0.86910 Accuracy: 0.52814
Epoch: 150 Batch:   0 Loss: 0.85274 Accuracy: 0.00000
Epoch: 151 Batch:   0 Loss: 0.85220 Accuracy: 0.00000
Epoch: 152 Batch:   0 Loss: 0.85167 Accuracy: 0.00000
Epoch: 153 Batch:   0 Loss: 0.85115 Accuracy: 0.00000
Epoch: 154 Batch:   0 Loss: 0.85065 Accuracy: 0.00000
Epoch: 155 Batch:   0 Loss: 0.85017 Accuracy: 0.00000
Epoch: 156 Batch:   0 Loss: 0.84969 Accuracy: 0.00000
Epoch: 157 Batch:   0 Loss: 0.84922 Accuracy: 0.00000
Epoch: 158 Batch:   0 Loss: 0.84876 Accuracy: 0.00000
Epoch: 159 Batch:   0 Loss: 0.84831 Accuracy: 0.00000
Test Loss: 0.86741 Accuracy: 0.52863
Epoch: 1

Epoch: 286 Batch:   0 Loss: 0.81405 Accuracy: 0.18750
Epoch: 287 Batch:   0 Loss: 0.81376 Accuracy: 0.18750
Epoch: 288 Batch:   0 Loss: 0.81348 Accuracy: 0.18750
Epoch: 289 Batch:   0 Loss: 0.81319 Accuracy: 0.21875
Test Loss: 0.86072 Accuracy: 0.55124
Epoch: 290 Batch:   0 Loss: 0.81291 Accuracy: 0.21875
Epoch: 291 Batch:   0 Loss: 0.81262 Accuracy: 0.21875
Epoch: 292 Batch:   0 Loss: 0.81233 Accuracy: 0.21875
Epoch: 293 Batch:   0 Loss: 0.81204 Accuracy: 0.25000
Epoch: 294 Batch:   0 Loss: 0.81175 Accuracy: 0.25000
Epoch: 295 Batch:   0 Loss: 0.81146 Accuracy: 0.25000
Epoch: 296 Batch:   0 Loss: 0.81116 Accuracy: 0.25000
Epoch: 297 Batch:   0 Loss: 0.81087 Accuracy: 0.25000
Epoch: 298 Batch:   0 Loss: 0.81057 Accuracy: 0.28125
Epoch: 299 Batch:   0 Loss: 0.81028 Accuracy: 0.28125
Test Loss: 0.85991 Accuracy: 0.55445
Epoch: 300 Batch:   0 Loss: 0.80998 Accuracy: 0.28125
Epoch: 301 Batch:   0 Loss: 0.80968 Accuracy: 0.28125
Epoch: 302 Batch:   0 Loss: 0.80939 Accuracy: 0.28125
Epoch: 3

[I 2023-11-17 14:51:23,448] Trial 19 finished with value: 0.5906714917571371 and parameters: {'REG_W': 3.4256544426719675e-06, 'REG_B': 0.0012195362651619677, 'REG_Z': 4.640666946458155e-05, 'SPAR_W': 0.9249592321637052, 'SPAR_B': 0.8062760865390182, 'SPAR_Z': 0.9531998601449847, 'LEARNING_RATE': 0.0002936820967518912, 'NUM_EPOCHS': 412}. Best is trial 10 with value: 0.7833735424205871.


In [14]:
study.best_trials

[FrozenTrial(number=10, state=TrialState.COMPLETE, values=[0.7833735424205871], datetime_start=datetime.datetime(2023, 11, 17, 13, 58, 41, 309742), datetime_complete=datetime.datetime(2023, 11, 17, 14, 4, 12, 447046), params={'REG_W': 3.1651209759814418e-06, 'REG_B': 1.5029243127284535e-05, 'REG_Z': 4.9678837029664995e-05, 'SPAR_W': 0.9694619549595722, 'SPAR_B': 0.8063620879846112, 'SPAR_Z': 0.9979067852308309, 'LEARNING_RATE': 0.0001202698283373907, 'NUM_EPOCHS': 484}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'REG_W': FloatDistribution(high=5e-06, log=False, low=2e-06, step=None), 'REG_B': FloatDistribution(high=0.01, log=False, low=0.0, step=None), 'REG_Z': FloatDistribution(high=5e-05, log=False, low=2e-05, step=None), 'SPAR_W': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'SPAR_B': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'SPAR_Z': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'LEARNING_RATE': FloatDistri

In [15]:
PROJECTION_DIM = 5 #d^
NUM_PROTOTYPES = 40 #m
REG_W = 3.1545272147130644e-06
REG_B = 0.00025738817748430687
REG_Z = 3.534809578054365e-05
SPAR_W = 0.6622823978261199
SPAR_B = 0.651918182305612
SPAR_Z = 0.6197401480486052
LEARNING_RATE = 0.00029791501581002825
NUM_EPOCHS = 595
BATCH_SIZE = 32
GAMMA = 0.007586

# Model Training

In [16]:

# Setup input and train protoNN


X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()

trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=10)


Epoch:   0 Batch:   0 Loss: 1.96731 Accuracy: 0.00000
Epoch:   1 Batch:   0 Loss: 2.25109 Accuracy: 0.00000
Epoch:   2 Batch:   0 Loss: 1.97812 Accuracy: 0.00000
Epoch:   3 Batch:   0 Loss: 1.88018 Accuracy: 0.00000
Epoch:   4 Batch:   0 Loss: 1.84649 Accuracy: 0.00000
Epoch:   5 Batch:   0 Loss: 1.83512 Accuracy: 0.00000
Epoch:   6 Batch:   0 Loss: 1.83033 Accuracy: 0.00000
Epoch:   7 Batch:   0 Loss: 1.82631 Accuracy: 0.00000
Epoch:   8 Batch:   0 Loss: 1.82104 Accuracy: 0.00000
Epoch:   9 Batch:   0 Loss: 1.81396 Accuracy: 0.00000
Test Loss: 0.96558 Accuracy: 0.50020
Epoch:  10 Batch:   0 Loss: 1.80509 Accuracy: 0.00000
Epoch:  11 Batch:   0 Loss: 1.79451 Accuracy: 0.00000
Epoch:  12 Batch:   0 Loss: 1.78257 Accuracy: 0.00000
Epoch:  13 Batch:   0 Loss: 1.76965 Accuracy: 0.00000
Epoch:  14 Batch:   0 Loss: 1.75536 Accuracy: 0.00000
Epoch:  15 Batch:   0 Loss: 1.74069 Accuracy: 0.00000
Epoch:  16 Batch:   0 Loss: 1.72541 Accuracy: 0.00000
Epoch:  17 Batch:   0 Loss: 1.70923 Accuracy:

Epoch: 143 Batch:   0 Loss: 0.59260 Accuracy: 1.00000
Epoch: 144 Batch:   0 Loss: 0.59133 Accuracy: 1.00000
Epoch: 145 Batch:   0 Loss: 0.59007 Accuracy: 1.00000
Epoch: 146 Batch:   0 Loss: 0.58889 Accuracy: 1.00000
Epoch: 147 Batch:   0 Loss: 0.58771 Accuracy: 1.00000
Epoch: 148 Batch:   0 Loss: 0.58655 Accuracy: 1.00000
Epoch: 149 Batch:   0 Loss: 0.58539 Accuracy: 1.00000
Test Loss: 0.76412 Accuracy: 0.67794
Epoch: 150 Batch:   0 Loss: 0.58414 Accuracy: 1.00000
Epoch: 151 Batch:   0 Loss: 0.58292 Accuracy: 1.00000
Epoch: 152 Batch:   0 Loss: 0.58173 Accuracy: 1.00000
Epoch: 153 Batch:   0 Loss: 0.58055 Accuracy: 1.00000
Epoch: 154 Batch:   0 Loss: 0.57940 Accuracy: 1.00000
Epoch: 155 Batch:   0 Loss: 0.57824 Accuracy: 1.00000
Epoch: 156 Batch:   0 Loss: 0.57713 Accuracy: 1.00000
Epoch: 157 Batch:   0 Loss: 0.57601 Accuracy: 1.00000
Epoch: 158 Batch:   0 Loss: 0.57492 Accuracy: 1.00000
Epoch: 159 Batch:   0 Loss: 0.57382 Accuracy: 1.00000
Test Loss: 0.76141 Accuracy: 0.68317
Epoch: 1

Epoch: 286 Batch:   0 Loss: 0.48397 Accuracy: 1.00000
Epoch: 287 Batch:   0 Loss: 0.48346 Accuracy: 1.00000
Epoch: 288 Batch:   0 Loss: 0.48295 Accuracy: 1.00000
Epoch: 289 Batch:   0 Loss: 0.48245 Accuracy: 1.00000
Test Loss: 0.73289 Accuracy: 0.72416
Epoch: 290 Batch:   0 Loss: 0.48194 Accuracy: 1.00000
Epoch: 291 Batch:   0 Loss: 0.48146 Accuracy: 1.00000
Epoch: 292 Batch:   0 Loss: 0.48093 Accuracy: 1.00000
Epoch: 293 Batch:   0 Loss: 0.48042 Accuracy: 1.00000
Epoch: 294 Batch:   0 Loss: 0.47992 Accuracy: 1.00000
Epoch: 295 Batch:   0 Loss: 0.47937 Accuracy: 1.00000
Epoch: 296 Batch:   0 Loss: 0.47885 Accuracy: 1.00000
Epoch: 297 Batch:   0 Loss: 0.47834 Accuracy: 1.00000
Epoch: 298 Batch:   0 Loss: 0.47782 Accuracy: 1.00000
Epoch: 299 Batch:   0 Loss: 0.47732 Accuracy: 1.00000
Test Loss: 0.73106 Accuracy: 0.72586
Epoch: 300 Batch:   0 Loss: 0.47683 Accuracy: 1.00000
Epoch: 301 Batch:   0 Loss: 0.47636 Accuracy: 1.00000
Epoch: 302 Batch:   0 Loss: 0.47588 Accuracy: 1.00000
Epoch: 3

Epoch: 429 Batch:   0 Loss: 0.42628 Accuracy: 1.00000
Test Loss: 0.71115 Accuracy: 0.74384
Epoch: 430 Batch:   0 Loss: 0.42595 Accuracy: 1.00000
Epoch: 431 Batch:   0 Loss: 0.42562 Accuracy: 1.00000
Epoch: 432 Batch:   0 Loss: 0.42528 Accuracy: 1.00000
Epoch: 433 Batch:   0 Loss: 0.42495 Accuracy: 1.00000
Epoch: 434 Batch:   0 Loss: 0.42465 Accuracy: 1.00000
Epoch: 435 Batch:   0 Loss: 0.42431 Accuracy: 1.00000
Epoch: 436 Batch:   0 Loss: 0.42396 Accuracy: 1.00000
Epoch: 437 Batch:   0 Loss: 0.42363 Accuracy: 1.00000
Epoch: 438 Batch:   0 Loss: 0.42330 Accuracy: 1.00000
Epoch: 439 Batch:   0 Loss: 0.42296 Accuracy: 1.00000
Test Loss: 0.70998 Accuracy: 0.74505
Epoch: 440 Batch:   0 Loss: 0.42264 Accuracy: 1.00000
Epoch: 441 Batch:   0 Loss: 0.42231 Accuracy: 1.00000
Epoch: 442 Batch:   0 Loss: 0.42198 Accuracy: 1.00000
Epoch: 443 Batch:   0 Loss: 0.42165 Accuracy: 1.00000
Epoch: 444 Batch:   0 Loss: 0.42133 Accuracy: 1.00000
Epoch: 445 Batch:   0 Loss: 0.42100 Accuracy: 1.00000
Epoch: 4

Epoch: 571 Batch:   0 Loss: 0.38455 Accuracy: 1.00000
Epoch: 572 Batch:   0 Loss: 0.38429 Accuracy: 1.00000
Epoch: 573 Batch:   0 Loss: 0.38402 Accuracy: 1.00000
Epoch: 574 Batch:   0 Loss: 0.38377 Accuracy: 1.00000
Epoch: 575 Batch:   0 Loss: 0.38351 Accuracy: 1.00000
Epoch: 576 Batch:   0 Loss: 0.38325 Accuracy: 1.00000
Epoch: 577 Batch:   0 Loss: 0.38299 Accuracy: 1.00000
Epoch: 578 Batch:   0 Loss: 0.38274 Accuracy: 1.00000
Epoch: 579 Batch:   0 Loss: 0.38248 Accuracy: 1.00000
Test Loss: 0.69762 Accuracy: 0.75399
Epoch: 580 Batch:   0 Loss: 0.38223 Accuracy: 1.00000
Epoch: 581 Batch:   0 Loss: 0.38197 Accuracy: 1.00000
Epoch: 582 Batch:   0 Loss: 0.38172 Accuracy: 1.00000
Epoch: 583 Batch:   0 Loss: 0.38147 Accuracy: 1.00000
Epoch: 584 Batch:   0 Loss: 0.38121 Accuracy: 1.00000
Epoch: 585 Batch:   0 Loss: 0.38096 Accuracy: 1.00000
Epoch: 586 Batch:   0 Loss: 0.38071 Accuracy: 1.00000
Epoch: 587 Batch:   0 Loss: 0.38046 Accuracy: 1.00000
Epoch: 588 Batch:   0 Loss: 0.38020 Accuracy:

# Model Evaluation

In [17]:
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
pred = sess.run(protoNN.predictions, feed_dict={X: x_test, Y: y_test})
# W, B, Z are tensorflow graph nodes
W, B, Z, _ = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]                       
nnz, size, sparse = getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)

Final test accuracy 0.7552272
Model size constraint (Bytes):  9580
Number of non-zeros:  2395


In [18]:
from sklearn.metrics import confusion_matrix,classification_report
y_test = np.argmax(y_test,axis=1)
print (confusion_matrix(y_test,pred))
print (classification_report(y_test,pred,digits=5))

[[2946 2028]
 [ 407 4567]]
              precision    recall  f1-score   support

           0    0.87862   0.59228   0.70758      4974
           1    0.69249   0.91817   0.78952      4974

    accuracy                        0.75523      9948
   macro avg    0.78556   0.75523   0.74855      9948
weighted avg    0.78556   0.75523   0.74855      9948



In [19]:
sensitivity = confusion_matrix(y_test,pred)[1][1]/(confusion_matrix(y_test,pred)[1][1] + confusion_matrix(y_test,pred)[1][0])
sensitivity

0.9181745074386811

In [20]:
specificity = confusion_matrix(y_test,pred)[0][0]/(confusion_matrix(y_test,pred)[0][0] + confusion_matrix(y_test,pred)[0][1])
specificity

0.5922798552472859

In [21]:
# accuracy = (TP + TN) / (TP + TN + FP + FN) 
# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)
# sensitivity = TP / (TP + FN) 
# specificity = TN / (TN + FP) 
# positive predictive value (PPV) = TP / (TP + FP)